## Prometheus Metrics
- Reference: https://nvidia.github.io/TensorRT-LLM/examples/prometheus_metrics.html

- Modified script for smoke tests measurement via Prometheus metrics

```
podman ps
CONTAINER ID  IMAGE                              COMMAND               CREATED        STATUS        PORTS       NAMES
2c0e74b44b1e  docker.io/vllm/vllm-openai:latest  -lc vllm serve /m...  7 minutes ago  Up 7 minutes              brave_wozniak
```

In [ ]:
%%bash
curl -s http://localhost:8002/v1/models | jq

In [ ]:
# !curl -s http://localhost:8002/metrics | grep -E '^(vllm|trtllm|# HELP|# TYPE)' | head -n 60

In [ ]:
from urllib.request import urlopen

from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(
    base_url="http://localhost:8002/v1",
    api_key="tensorrt_llm",
)

# Prometheus metric prefix used by TensorRT-LLM
METRIC_PREFIX = "vllm:"

# Base URL for the metrics endpoint
METRICS_URL = "http://localhost:8002/metrics"



In [ ]:
def fetch_metrics() -> dict | None:
    """Fetch metrics from the Prometheus endpoint."""
    try:
        response = urlopen(METRICS_URL)
        if response.status == 200:
            return response.read().decode("utf-8")
        else:
            print(f"Error fetching metrics: HTTP {response.status}")
            return None
    except Exception as e:
        print(f"Error fetching metrics: {e}")
        return None

In [ ]:
def parse_and_display_metrics(metrics_data: dict) -> None:
    """Parse and display relevant TensorRT-LLM metrics."""
    if not metrics_data:
        return

    print("\n" + "=" * 80)
    print("TensorRT-LLM Prometheus Metrics")
    print("=" * 80)

    # Define metrics to display with descriptions
    metrics_of_interest = {
        f"{METRIC_PREFIX}request_success_total": "Total successful requests",
        f"{METRIC_PREFIX}e2e_request_latency_seconds": "End-to-end request latency",
        f"{METRIC_PREFIX}time_to_first_token_seconds": "Time to first token",
        f"{METRIC_PREFIX}request_queue_time_seconds": "Request queue time",
        f"{METRIC_PREFIX}kv_cache_hit_rate": "KV cache hit rate",
        f"{METRIC_PREFIX}kv_cache_utilization": "KV cache utilization",
        f"{METRIC_PREFIX}prefix_cache_queries_total": "Prefix cache queries (tokens)",
        f"{METRIC_PREFIX}prefix_cache_hits_total": "Prefix cache hits (cached tokens)",
        f"{METRIC_PREFIX}external_prefix_cache_queries_total": "External prefix cache queries",
        f"{METRIC_PREFIX}external_prefix_cache_hits_total": "External prefix cache hits",

    }

    found_metrics = []
    missing_metrics = []

    for metric_name, description in metrics_of_interest.items():
        if metric_name in metrics_data:
            found_metrics.append((metric_name, description))
        else:
            missing_metrics.append((metric_name, description))

    # Display found metrics
    if found_metrics:
        print("\n✓ Available Metrics:")
        print("-" * 80)
        for metric_name, description in found_metrics:
            # Extract the metric lines from the data
            lines = [
                line
                for line in metrics_data.split("\n")
                if line.startswith(metric_name) and not line.startswith("#")
            ]
            print(f"\n{description} ({metric_name}):")
            for line in lines:
                print(f"  {line}")

    # Display missing metrics
    if missing_metrics:
        print("\n✗ Not Yet Available:")
        print("-" * 80)
        for metric_name, description in missing_metrics:
            print(f"  {description} ({metric_name})")

    print("\n" + "=" * 80)

In [ ]:
def main():
    print("Prometheus Metrics Example")
    print("=" * 80)
    print("This script will:")
    print("1. Send several completion requests to a running TensorRT-LLM server")
    print(
        "2. After each response, fetch and display Prometheus metrics from the /prometheus/metrics endpoint"
    )
    print()

    # Auto-detect first available model from the OpenAI-compatible /v1/models endpoint
    model_id = client.models.list().data[0].id

    # Make several completion requests to generate metrics
    print("Sending completion requests...")
    num_requests = 10
    for i in range(num_requests):
        try:
            response = client.completions.create(
                model=model_id,
                prompt=(
                    f"Hello, this is request {i + 1}. "
                    "Use your greatest imagination in this request. Tell me a lot about"
                ),
                max_tokens=1000,
                stream=False,
            )
            print(
                f"  Request {i + 1}/{num_requests} completed. Response: {response.choices[0].text[:50]}..."
            )

            # Fetch and display metrics after each response
            print(f"\n  Fetching metrics after request {i + 1}...")
            metrics_data = fetch_metrics()
            if metrics_data:
                parse_and_display_metrics(metrics_data)
            else:
                print("  ✗ Failed to fetch metrics")
            print()
        except Exception as e:
            print(f"  Error on request {i + 1}: {e}")
    print("All requests completed.")

In [ ]:
if __name__ == "__main__":
    main()

#### TP2

```
rteixeira@elita:/mnt/elita/soundwave/playground$ nvidia-smi
Sun Mar  1 00:21:17 2026
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.119.02             Driver Version: 580.119.02     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L40S                    On  |   00000000:4A:00.0 Off |                    0 |
| N/A   40C    P0             82W /  350W |   43147MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
|   1  NVIDIA L40S                    On  |   00000000:61:00.0 Off |                    0 |
| N/A   40C    P0             86W /  350W |   43147MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
|   2  NVIDIA L40S                    On  |   00000000:CA:00.0 Off |                    0 |
| N/A   31C    P8             32W /  350W |       0MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
|   3  NVIDIA L40S                    On  |   00000000:E1:00.0 Off |                    0 |
| N/A   30C    P8             32W /  350W |       0MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+

+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI              PID   Type   Process name                        GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|    0   N/A  N/A          420305      C   VLLM::Worker_TP0                      43138MiB |
|    1   N/A  N/A          420306      C   VLLM::Worker_TP1                      43138MiB |
+-----------------------------------------------------------------------------------------+
```

### On container:

```bash
 podman run --rm -it --net=host   --hooks-dir=/usr/share/containers/oci/hooks.d   --security-opt=label=disable   --device nvidia.com/gpu=0   --device nvidia.com/gpu=1   -e HF_HOME=/mnt/elita/soundwave/hf_cache   -e VLLM_CACHE_DIR=/mnt/elita/soundwave/vllm_cache   -v /mnt/elita/soundwave:/mnt/elita/soundwave   --entrypoint /bin/bash   docker.io/vllm/vllm-openai:latest -lc   'vllm serve /mnt/elita/soundwave/models/llama3-70b-awq \
     --port 8002 \
     --tensor-parallel-size 2 \
     2>&1 | tee /mnt/elita/soundwave/logs/vllm_tp2_default_DEBUG_8002.log'
```

```
<...>

```